In [1]:
%%time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from scipy import spatial
from datetime import timedelta
import seaborn as sns


#df2 = pd.read_csv('D:\data from MnM Samdal raw\Samdal_power_1s_no_outliers_after_only_window.csv',  parse_dates=[0], index_col = 0, low_memory=False)

# upload file with outliers removed with window method in power and speed
df = pd.read_csv('D:\data from MnM Samdal raw\Samdal_power_1s_no_outliers_after_window_30s_20perc_spd10_pow.csv',  parse_dates=[0], index_col = 0)
df2 = df.copy(deep=True)

Wall time: 1min 1s


In [ ]:
new_index = pd.date_range('4/20/2011', '10/10/2011' , freq='1s')
df = df2.reindex(new_index)

In [2]:
# plot to see power curve

#plt.plot(outliers_spd.speed, marker='o', label = 'data points above the threshold wind speed (erroneous data)')


# plt.scatter(df4.speed, df4.power, c='lightgrey', label='data before outlier removal')
# plt.scatter(outliers_spd.speed, outliers_spd.power, c='r', s=50, label='detected outliers')

#plt.scatter(df.speed, df.power, c='g', s=50, label='after outliers removal')
plt.scatter(df.speed, df.power, c='g', s=50)



plt.xlabel('wind speed [m/s]', fontsize = 26)
plt.ylabel('power [kW]', fontsize = 26)
plt.xlim(-1,)
plt.ylim(-100,)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=22)
#plt.legend(fontsize=22, bbox_to_anchor=(0.6, 1.01), loc=3)

# plt.title('Detected outliers, 1s interval, threshold = {:.0f} [m/s] (= {:.0%} of Vmax), window = {:.0f}s '.format(thresh_differ_spd, percent_spd, win_size) ,
#          fontsize = 28)


plt.show()

In [ ]:
# plot where points are between 15.5 and 14.5 kW





In [2]:
# remove theoretical outliers


# zero power
df.power = df.power.drop(df[(df.speed > 10) & (df.power >=0) & (df.power <=150)].index)

# above cut-out speed
df.power = df.power.drop(df[(df.speed > 24) ].index)

#the 1500 line
#df.power = df.power.drop(df[(df.power >1450) & (df.power <1550) & (df.speed<6) & (df.speed >12)].index)
#df.power = df.power.drop(df[(df.power >1450) & (df.power <1550) & (df.speed<6) & (df.speed >12) ].index)


# above the curve

#define linear equation to remove points above that line


# Calculate the slope between the two points

x1 = 0.77
y1 = 145

# point defined from nominal WS and WP
x2 = 5.15
y2 = 1150

m = (y2 - y1)/(x2 - x1)

b = y2 - m*x2

x  = df.speed

y = m*x + b

# TODO this method drops only values in column. NOT the whole row
#df = df.drop(df[(df.power > y) & (df.power >=0) & (df.power <1)].index)
df.power = df.power.drop(df[(df.power > y) & (df.power <1300)& (df.power >10)].index)


# ##################

# # points below the curve

# #define linear equation to remove points above that line


# # Calculate the slope between the two points
#
x11 = 8
y11 = 0

# point defined from nominal WS and WP
x22 = 12.5
y22 = 1900

m1 = (y22 - y11)/(x22 - x11)

b1 = y22 - m1*x22

x  = df.speed

y2 = m1*x + b1



#df = df.drop(df[(df.power < y2) & (df.power<2300)].index)
df.power = df.power.drop(df[(df.power < y2) & (df.power<2300)].index)


######################


##################

# points aboe the upper part of the curve

#define linear equation to remove points above that line


# Calculate the slope between the two points
#
# x111 = 8.125
# y111 = 2500

x111 = 7
y111 = 2500

# point defined from nominal WS and WP
# x222 = 6.37
# y222 = 1520

x222 = 0
y222 = 0

m11 = (y222 - y111)/(x222 - x111)

b11 = y222 - m11*x222

x  = df.speed

y3 = m11*x + b11


df.power = df.power.drop(df[(df.power > y3) & (df.power>1300)].index)





In [3]:
%%time
# remove the 'line' outliers

# quick workaround
df.power = df.power.drop(df[(df.power > 1500) & (df.power<1520) & (df.speed<8.4)].index)

df.power = df.power.drop(df[(df.power<16) & (df.speed>3.5)].index)
#df.power = df.power.drop(df[(df.power<23)].index)


Wall time: 10.8 s


In [12]:
######################

plt.legend(fontsize=22)
plt.xlabel('wind speed [m/s]', fontsize = 26)
plt.ylabel('power [kW]', fontsize = 26)
plt.xlim(-1,30)
plt.ylim(-100,3000)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title('Power curve after removing outliers with "window" method and removing the remaining outliers', fontsize = 26)

plt.scatter(df2.speed, df2.power, s=30, label='outliers', c='r')
plt.scatter(df.speed, df.power, s=30, label='correct data', c='mediumblue')
plt.legend(fontsize=22)

print('len wo outliers', len(df))
# print(len(df.power))
# print(len(df.speed))
#
# df.power1 = df.power1.interpolate()
# df.power2 = df.power2.interpolate()
plt.show()

#df.to_csv('Samdal_power_1s_wo_outliers.csv')



C:\Anaconda3\lib\site-packages\matplotlib\axes\_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


len wo outliers 10855329


In [11]:
print((len(df2.power.dropna())-len(df.power.dropna()))/len(df2.power.dropna()))
print(len(df.power.dropna()))
print(len(df2.power.dropna()))

0.07531803043463722
10037727
10855329


In [17]:
# save the file for further use

df.to_csv('D:\data from MnM Samdal raw\Samdal_power_1s_no_outliers_after_window_and_theoret.csv')

In [19]:
df1 = df.resample('1min').mean()
df3 = df.resample('10min').mean()
df5 = df.resample('30min').mean()
df6 = df.resample('60min').mean()


df1.to_csv('D:\data from MnM Samdal raw\Samdal_power_1min_new_wo.csv')
df3.to_csv('D:\data from MnM Samdal raw\Samdal_power_10min_new_wo.csv')
df5.to_csv('D:\data from MnM Samdal raw\Samdal_power_30min_new_wo.csv')
df6.to_csv('D:\data from MnM Samdal raw\Samdal_power_1h_new_wo.csv')



In [ ]:
# calculate differences for each file and save to csv 

next_powers = df.power[1:].values 

df['next_power']=  next_powers.append(0)    




df['difference'] = df['next_power'] - df['power']

######################

# merge them together into a fourth column that indicates what you want: 
# the column is True when the row is before, on, or after a nan value.

missing = df.difference.isnull()
df['is_nan'] = missing
df['before_nan'] = np.append(missing[1:].values, False)
df['after_nan'] = np.insert(missing[:-1].values, 0, False)

 # a column which says 'True' to rows which should be removed
df['around_nan'] = df.is_nan | df.before_nan | df.after_nan  

#mark('looped')

mark('time-inefficient loop done')

# remove False values from the df.around_nan

def take_values():
    for x in range(len(df.around_nan)):
        
        if df.around_nan.ix[x] == False:
            yield df.difference.ix[x]
        else:
            yield np.nan

#values = take_values()            
            
list_cleaned_values = list(take_values())
df['diff_cleaned'] =  list_cleaned_values  

df.to_csv('D:\data from MnM Samdal raw\Samdal_power_1s_differences_sdfs.csv')